In [16]:
# File Directory 
import glob
import os
from os.path import isdir, join
from pathlib import Path

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
import librosa

# Dimension Reduction
from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display
'''import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls'''

# Data Pre-processing
import pandas as pd
from sklearn.model_selection import KFold
import soundfile

# Deep Learning
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K

# Configuration
#py.init_notebook_mode(connected=True)

from keras.activations import relu, softmax
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras import losses, models, optimizers
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

%matplotlib inline

Using TensorFlow backend.


In [4]:
samples=[]
sample_rates=[]
labels = []
sample_slice_iteration = 0
gunshot_aggregator = {}
glassbreak_aggregator = {}

gunshot_sound_dir = "C:\\Users\\hosle\\Documents\\_REU2019\\gunshot\\"

for file in os.listdir(gunshot_sound_dir):
    if file.endswith(".wav"):
        try:
            sample, sample_rate = librosa.load(gunshot_sound_dir + file)
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                label = 2
                gunshot_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.25:
                    label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
        except:
            sample, sample_rate = soundfile.read(gunshot_sound_dir + file)
            #print("Gunshot sound unrecognized by Librosa:", sample)
            pass
        
glassbreak_sound_dir = "C:\\Users\\hosle\\Documents\\_REU2019\\glassbreak\\"

print("...Switching to glassbreak sounds...")

for file in os.listdir(glassbreak_sound_dir):
    if file.endswith(".wav"):
        try:
            sample, sample_rate = librosa.load(glassbreak_sound_dir + file)
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                label = 1
                glassbreak_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.5:
                    label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
        except:
            sample, sample_rate = soundfile.read(glassbreak_sound_dir + file)
            print("Glassbreak sound unrecognized by Librosa:", sample)
            pass

...Switching to glassbreak sounds...


In [5]:
#read in the csv file of descriptors for all other urban sounds
sound_types = pd.read_csv("C:\\Users\\hosle\\Documents\\_REU2019\\_project\\gunshot_detection\\train.csv")
print(sound_types.loc[0,'Class'])

urban_aggregator = {}
j=0
#read in all of the wav files similar to above
urban_sound_dir = "C:\\Users\\hosle\\Documents\\_REU2019\\_project\\gunshot_detection\\Train\\"
print(os.listdir(urban_sound_dir))

for file in os.listdir(urban_sound_dir):
    if file.endswith(".wav"):
        try:
            sample, sample_rate = librosa.load(urban_sound_dir + file)
            print("librosa read the file ok #" + str(j))
            for i in range(0, sample.size - 44100, 44100):
                sample_slice = sample[i : i + 44100]
                if(sound_types.loc[j, 'Class'] == "gun_shot"):
                    label = 2
                else:
                    label = 0
                urban_aggregator[sample_slice_iteration] = np.max(abs(sample_slice))
                sample_slice_iteration += 1
                if np.max(abs(sample_slice)) < 0.25:
                    label = 0

                samples.append(sample_slice)
                sample_rates.append(sample_rate)
                labels.append(label)
            j +=1
        except:
            sample, sample_rate = soundfile.read(urban_sound_dir + file)
            print("Urban sound unrecognized by Librosa:", sample)
            pass


siren
['0.wav', '1.wav', '10.wav', '100.wav', '1000.wav', '1001.wav', '1003.wav', '1004.wav', '1006.wav', '1007.wav', '1008.wav', '101.wav', '1014.wav', '1015.wav', '1017.wav', '1018.wav', '1021.wav', '1022.wav', '1024.wav', '1025.wav', '1026.wav', '1027.wav', '1028.wav', '1029.wav', '103.wav', '1030.wav', '1032.wav', '1034.wav', '1036.wav', '1038.wav', '104.wav', '1040.wav', '1042.wav', '1043.wav', '1044.wav', '1045.wav', '1047.wav', '1048.wav', '1049.wav', '105.wav', '1051.wav', '1052.wav', '1054.wav', '1055.wav', '1056.wav', '1058.wav', '1059.wav', '1061.wav', '1062.wav', '1063.wav', '1066.wav', '1068.wav', '1069.wav', '1070.wav', '1071.wav', '1072.wav', '1074.wav', '1076.wav', '1077.wav', '1078.wav', '1079.wav', '1080.wav', '1082.wav', '1083.wav', '1085.wav', '1087.wav', '1088.wav', '1089.wav', '1090.wav', '1091.wav', '1092.wav', '1096.wav', '1098.wav', '11.wav', '1100.wav', '1105.wav', '1106.wav', '1112.wav', '1113.wav', '1115.wav', '1116.wav', '1118.wav', '1119.wav', '1120.wav', 

librosa read the file ok #1
librosa read the file ok #2
librosa read the file ok #3
librosa read the file ok #4
librosa read the file ok #5
librosa read the file ok #6
librosa read the file ok #7
librosa read the file ok #8
librosa read the file ok #9
librosa read the file ok #10
librosa read the file ok #11
librosa read the file ok #12
librosa read the file ok #13
librosa read the file ok #14
librosa read the file ok #15
librosa read the file ok #16
librosa read the file ok #17
librosa read the file ok #18
librosa read the file ok #19
librosa read the file ok #20
librosa read the file ok #21
librosa read the file ok #22
librosa read the file ok #23
librosa read the file ok #24
librosa read the file ok #25
librosa read the file ok #26
librosa read the file ok #27
librosa read the file ok #28
librosa read the file ok #29
librosa read the file ok #30
librosa read the file ok #31
librosa read the file ok #32
librosa read the file ok #33
librosa read the file ok #34
librosa read the file o

librosa read the file ok #280
librosa read the file ok #281
librosa read the file ok #282
librosa read the file ok #283
librosa read the file ok #284
librosa read the file ok #285
librosa read the file ok #286
librosa read the file ok #287
librosa read the file ok #288
librosa read the file ok #289
librosa read the file ok #290
librosa read the file ok #291
librosa read the file ok #292
librosa read the file ok #293
librosa read the file ok #294
librosa read the file ok #295
librosa read the file ok #296
librosa read the file ok #297
librosa read the file ok #298
librosa read the file ok #299
librosa read the file ok #300
librosa read the file ok #301
librosa read the file ok #302
librosa read the file ok #303
librosa read the file ok #304
librosa read the file ok #305
librosa read the file ok #306
librosa read the file ok #307
librosa read the file ok #308
librosa read the file ok #309
librosa read the file ok #310
librosa read the file ok #311
librosa read the file ok #312
librosa re

librosa read the file ok #555
librosa read the file ok #556
librosa read the file ok #557
librosa read the file ok #558
librosa read the file ok #559
librosa read the file ok #560
librosa read the file ok #561
librosa read the file ok #562
librosa read the file ok #563
librosa read the file ok #564
librosa read the file ok #565
librosa read the file ok #566
librosa read the file ok #567
librosa read the file ok #568
librosa read the file ok #569
librosa read the file ok #570
librosa read the file ok #571
librosa read the file ok #572
librosa read the file ok #573
librosa read the file ok #574
librosa read the file ok #575
librosa read the file ok #576
librosa read the file ok #577
librosa read the file ok #578
librosa read the file ok #579
librosa read the file ok #580
librosa read the file ok #581
librosa read the file ok #582
librosa read the file ok #583
librosa read the file ok #584
librosa read the file ok #585
librosa read the file ok #586
librosa read the file ok #587
librosa re

librosa read the file ok #830
librosa read the file ok #831
librosa read the file ok #832
librosa read the file ok #833
librosa read the file ok #834
librosa read the file ok #835
librosa read the file ok #836
librosa read the file ok #837
librosa read the file ok #838
librosa read the file ok #839
librosa read the file ok #840
librosa read the file ok #841
librosa read the file ok #842
librosa read the file ok #843
librosa read the file ok #844
librosa read the file ok #845
librosa read the file ok #846
librosa read the file ok #847
librosa read the file ok #848
librosa read the file ok #849
librosa read the file ok #850
librosa read the file ok #851
librosa read the file ok #852
librosa read the file ok #853
librosa read the file ok #854
librosa read the file ok #855
librosa read the file ok #856
librosa read the file ok #857
librosa read the file ok #858
librosa read the file ok #859
librosa read the file ok #860
librosa read the file ok #861
librosa read the file ok #862
librosa re

...

librosa read the file ok #4283
librosa read the file ok #4284
librosa read the file ok #4285
librosa read the file ok #4286
librosa read the file ok #4287
librosa read the file ok #4288
librosa read the file ok #4289
librosa read the file ok #4290
librosa read the file ok #4291
librosa read the file ok #4292
librosa read the file ok #4293
librosa read the file ok #4294
librosa read the file ok #4295
librosa read the file ok #4296
librosa read the file ok #4297
librosa read the file ok #4298
librosa read the file ok #4299
librosa read the file ok #4300
librosa read the file ok #4301
librosa read the file ok #4302
librosa read the file ok #4303
librosa read the file ok #4304
librosa read the file ok #4305
librosa read the file ok #4306
librosa read the file ok #4307
librosa read the file ok #4308
librosa read the file ok #4309
librosa read the file ok #4310
librosa read the file ok #4311
librosa read the file ok #4312
librosa read the file ok #4313
librosa read the file ok #4314
librosa 

librosa read the file ok #4549
librosa read the file ok #4550
librosa read the file ok #4551
librosa read the file ok #4552
librosa read the file ok #4553
librosa read the file ok #4554
librosa read the file ok #4555
librosa read the file ok #4556
librosa read the file ok #4557
librosa read the file ok #4558
librosa read the file ok #4559
librosa read the file ok #4560
librosa read the file ok #4561
librosa read the file ok #4562
librosa read the file ok #4563
librosa read the file ok #4564
librosa read the file ok #4565
librosa read the file ok #4566
librosa read the file ok #4567
librosa read the file ok #4568
librosa read the file ok #4569
librosa read the file ok #4570
librosa read the file ok #4571
librosa read the file ok #4572
librosa read the file ok #4573
librosa read the file ok #4574
librosa read the file ok #4575
librosa read the file ok #4576
librosa read the file ok #4577
librosa read the file ok #4578
librosa read the file ok #4579
librosa read the file ok #4580
librosa 

librosa read the file ok #4815
librosa read the file ok #4816
librosa read the file ok #4817
librosa read the file ok #4818
librosa read the file ok #4819
librosa read the file ok #4820
librosa read the file ok #4821
librosa read the file ok #4822
librosa read the file ok #4823
librosa read the file ok #4824
librosa read the file ok #4825
librosa read the file ok #4826
librosa read the file ok #4827
librosa read the file ok #4828
librosa read the file ok #4829
librosa read the file ok #4830
librosa read the file ok #4831
librosa read the file ok #4832
librosa read the file ok #4833
librosa read the file ok #4834
librosa read the file ok #4835
librosa read the file ok #4836
librosa read the file ok #4837
librosa read the file ok #4838
librosa read the file ok #4839
librosa read the file ok #4840
librosa read the file ok #4841
librosa read the file ok #4842
librosa read the file ok #4843
librosa read the file ok #4844
librosa read the file ok #4845
librosa read the file ok #4846
librosa 

librosa read the file ok #5081
librosa read the file ok #5082
librosa read the file ok #5083
librosa read the file ok #5084
librosa read the file ok #5085
librosa read the file ok #5086
librosa read the file ok #5087
librosa read the file ok #5088
librosa read the file ok #5089
librosa read the file ok #5090
librosa read the file ok #5091
librosa read the file ok #5092
librosa read the file ok #5093
librosa read the file ok #5094
librosa read the file ok #5095
librosa read the file ok #5096
librosa read the file ok #5097
librosa read the file ok #5098
librosa read the file ok #5099
librosa read the file ok #5100
librosa read the file ok #5101
librosa read the file ok #5102
librosa read the file ok #5103
librosa read the file ok #5104
librosa read the file ok #5105
librosa read the file ok #5106
librosa read the file ok #5107
librosa read the file ok #5108
librosa read the file ok #5109
librosa read the file ok #5110
librosa read the file ok #5111
librosa read the file ok #5112
librosa 

librosa read the file ok #5346
librosa read the file ok #5347
librosa read the file ok #5348
librosa read the file ok #5349
librosa read the file ok #5350
librosa read the file ok #5351
librosa read the file ok #5352
librosa read the file ok #5353
librosa read the file ok #5354
librosa read the file ok #5355
librosa read the file ok #5356
librosa read the file ok #5357
librosa read the file ok #5358
librosa read the file ok #5359
librosa read the file ok #5360
librosa read the file ok #5361
librosa read the file ok #5362
librosa read the file ok #5363
librosa read the file ok #5364
librosa read the file ok #5365
librosa read the file ok #5366
librosa read the file ok #5367
librosa read the file ok #5368
librosa read the file ok #5369
librosa read the file ok #5370
librosa read the file ok #5371
librosa read the file ok #5372
librosa read the file ok #5373
librosa read the file ok #5374
librosa read the file ok #5375
librosa read the file ok #5376
librosa read the file ok #5377
librosa 

In [6]:
print(len(samples))
i=744
samp=samples[i]
sr=sample_rates[i]
print(np.max(abs(samp)))
print(labels[i])
ipd.Audio(samp, rate=sr)

5662
1.0389285
1


In [8]:
labels

[2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,


In [25]:
'''
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
'''

"\ndef log_specgram(audio, sample_rate, window_size=20,\n                 step_size=10, eps=1e-10):\n    nperseg = int(round(window_size * sample_rate / 1e3))\n    noverlap = int(round(step_size * sample_rate / 1e3))\n    freqs, times, spec = signal.spectrogram(audio,\n                                    fs=sample_rate,\n                                    window='hann',\n                                    nperseg=nperseg,\n                                    noverlap=noverlap,\n                                    detrend=False)\n    return freqs, times, np.log(spec.T.astype(np.float32) + eps)\n"

In [26]:
'''
i=15
samp=samples[i]
sr=sample_rates[i]

freqs, times, spectrogram = log_specgram(samp, sr)

fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ' + file)
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, 1,samp.size), samp)

ax2 = fig.add_subplot(212)
ax2.imshow(spectrogram.T, aspect='auto', origin='lower', 
           extent=[times.min(), times.max(), freqs.min(), freqs.max()])
ax2.set_yticks(freqs[::16])
ax2.set_xticks(times[::16])
ax2.set_title('Spectrogram of ' + file)
ax2.set_ylabel('Freqs in Hz')
ax2.set_xlabel('Seconds')
'''

"\ni=15\nsamp=samples[i]\nsr=sample_rates[i]\n\nfreqs, times, spectrogram = log_specgram(samp, sr)\n\nfig = plt.figure(figsize=(14, 8))\nax1 = fig.add_subplot(211)\nax1.set_title('Raw wave of ' + file)\nax1.set_ylabel('Amplitude')\nax1.plot(np.linspace(0, 1,samp.size), samp)\n\nax2 = fig.add_subplot(212)\nax2.imshow(spectrogram.T, aspect='auto', origin='lower', \n           extent=[times.min(), times.max(), freqs.min(), freqs.max()])\nax2.set_yticks(freqs[::16])\nax2.set_xticks(times[::16])\nax2.set_title('Spectrogram of ' + file)\nax2.set_ylabel('Freqs in Hz')\nax2.set_xlabel('Seconds')\n"

In [23]:
kf = KFold(n_splits=3, shuffle=True)
samples = np.array(samples)
labels = np.array(labels)
np.save("samples.npy", samples)
np.save("labels.npy", labels)
for train_index, test_index in kf.split(samples):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_wav, test_wav = samples[train_index], samples[test_index]
    train_label, test_label = labels[train_index], labels[test_index]

TRAIN: [   0    1    2 ... 5654 5658 5659] TEST: [   4   12   18 ... 5657 5660 5661]
TRAIN: [   1    4    5 ... 5657 5660 5661] TEST: [   0    2    3 ... 5651 5658 5659]
TRAIN: [   0    2    3 ... 5659 5660 5661] TEST: [   1    5    6 ... 5647 5653 5654]


In [24]:
# Parameters
lr = 0.001
generations = 20000
num_gens_to_wait = 250
batch_size = 32
drop_out_rate = 0.2
input_shape = (44100,1)

In [25]:
#For Conv1D add Channel
train_wav = np.array(train_wav)
test_wav = np.array(test_wav)
train_wav = train_wav.reshape(-1,44100,1)
test_wav = test_wav.reshape(-1,44100,1)
train_label = keras.utils.to_categorical(train_label, 3)
test_label = keras.utils.to_categorical(test_label, 3)

In [26]:
print(train_wav.shape)

(3775, 44100, 1)


In [27]:
input_tensor = Input(shape=input_shape)
nclass = 3

x = Convolution1D(16, 9, activation=relu, padding="valid")(input_tensor)
x = Convolution1D(16, 9, activation=relu, padding="valid")(x)
x = MaxPool1D(16)(x)
x = Dropout(rate=0.1)(x)

x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
x = MaxPool1D(4)(x)
x = Dropout(rate=0.1)(x)

x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
x = Convolution1D(32, 3, activation=relu, padding="valid")(x)
x = MaxPool1D(4)(x)
x = Dropout(rate=0.1)(x)

x = Convolution1D(256, 3, activation=relu, padding="valid")(x)
x = Convolution1D(256, 3, activation=relu, padding="valid")(x)
x = GlobalMaxPool1D()(x)
x = Dropout(rate=0.2)(x)

x = Dense(64, activation=relu)(x)
x = Dense(1028, activation=relu)(x)
output_tensor = Dense(nclass, activation=softmax)(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)
opt = optimizers.Adam(0.001)

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])

In [28]:
model_filename = '1Dcnngunglass.pkl' 
callbacks = [
    EarlyStopping(monitor='val_acc',
                  patience=10,
                  verbose=1,
                  mode='auto'),
    
    ModelCheckpoint(model_filename, monitor='val_acc',
                    verbose=1,
                    save_best_only=True,
                    mode='auto'),
]

In [29]:
model.fit(train_wav, train_label, 
          validation_data=[test_wav, test_label],
          batch_size=batch_size,
          callbacks = callbacks,
          epochs=100,
          verbose=1)

Train on 3775 samples, validate on 1887 samples
Epoch 1/100
3775/3775 [==============================] - 12s 3ms/step - loss: 0.2597 - acc: 0.9258 - val_loss: 0.2422 - val_acc: 0.9339

Epoch 00001: val_acc improved from -inf to 0.93393, saving model to 1Dcnngunglass.pkl
Epoch 2/100
3775/3775 [==============================] - 10s 3ms/step - loss: 0.2238 - acc: 0.9398 - val_loss: 0.2407 - val_acc: 0.9339

Epoch 00002: val_acc did not improve from 0.93393
Epoch 3/100
3775/3775 [==============================] - 10s 3ms/step - loss: 0.2129 - acc: 0.9397 - val_loss: 0.1888 - val_acc: 0.9357

Epoch 00003: val_acc improved from 0.93393 to 0.93570, saving model to 1Dcnngunglass.pkl
Epoch 4/100
3775/3775 [==============================] - 10s 3ms/step - loss: 0.1783 - acc: 0.9403 - val_loss: 0.1694 - val_acc: 0.9435

Epoch 00004: val_acc improved from 0.93570 to 0.94347, saving model to 1Dcnngunglass.pkl
Epoch 5/100
3775/3775 [==============================] - 10s 3ms/step - loss: 0.1484 - acc

In [26]:
Y_test_pred = model.predict(test_wav)
y_predicted_classes_test = Y_test_pred.argmax(axis=-1)
y_actual_classes_test= test_label.argmax(axis=-1)
wrong_examples = np.nonzero(y_predicted_classes_test != y_actual_classes_test)

(289,)


In [18]:
print(wrong_examples)

(array([  1,   5,  19,  41,  50,  52,  55,  57,  58,  64,  65,  67,  75,
        81, 112, 115, 116, 120, 121, 135, 138, 158, 193, 209, 224, 225,
       232, 235, 249, 256, 270, 284, 285, 287]),)


In [19]:
i=1
samp=np.reshape(test_wav[i],44100,)
sr=sample_rates[i]
print(y_test[i],Y_test_pred[i])
ipd.Audio(samp, rate=sr)

2 [0.554347   0.3448235  0.10082955]


In [ ]:
i=5
samp=np.reshape(test_wav[i],44100,)
sr=sample_rates[i]
print(y_test[i],Y_test_pred[i])
ipd.Audio(samp, rate=sr)

In [17]:
i=19
samp=np.reshape(test_wav[i],44100,)
sr=sample_rates[i]
print(y_test[i],Y_test_pred[i])
ipd.Audio(samp, rate=sr)

0 [0.9313842  0.06127589 0.00733979]


In [18]:
i=41
samp=np.reshape(test_wav[i],44100,)
sr=sample_rates[i]
print(y_test[i],Y_test_pred[i])
ipd.Audio(samp, rate=sr)

2 [5.9865852e-06 5.2870007e-07 9.9999344e-01]


In [19]:
i=50
samp=np.reshape(test_wav[i],44100,)
sr=sample_rates[i]
print(y_test[i],Y_test_pred[i])
ipd.Audio(samp, rate=sr)

2 [0.7274393  0.1795184  0.09304236]
